In [1]:
from google.colab import files
uploaded = files.upload()

Saving DataEx3.zip to DataEx3.zip


In [3]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['DataEx3.zip']), 'r')
data.extractall()

In [4]:
import os
import cv2
import numpy as np
import csv as csvD
import pandas as pd
import copy
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import shutil

In [8]:
dataframe = pd.read_csv(r"imagesBB.csv",sep=';',header=None)
dataframe.head(10)
#dataframe.shape[0]

,0,1,2,3,4,5
0,ROI_0_Psic_004Ev5.pdf_pg-27.jpg,476.82,149.87,509.74,206.96,NaN
1,ROI_0_Psic_008Ev1.pdf_pg-28.jpg,415.54,190.18,460.63,232.77,NaN
2,ROI_0_Psic_008Ev3.pdf_pg-18.jpg,331.33,160.62,362.71,209.64,NaN
3,ROI_0_Psic_010Ev1.pdf_pg-15.jpg,386.15,86.30,414.98,125.68,NaN
4,ROI_0_Psic_011Ev5.pdf_pg-25.jpg,401.89,169.01,438.30,234.25,NaN
5,ROI_0_Psic_33Ev4.pdf_pg-25.jpg,321.17,144.11,360.00,211.92,NaN
6,ROI_0_Psic_523Ev3.pdf_pg-27.jpg,381.22,140.25,416.95,201.05,NaN
7,ROI_90_Psic_025Ev4.pdf_pg-23.jpg,332.40,115.81,368.32,154.45,NaN
8,ROI_0_Psic_341Ev1-r.png,395.41,128.18,449.75,199.87,NaN
9,ROI_0_Psic_015Ev5.pdf_pg-27.jpg,424.23,201.66,460.75,240.17,NaN


In [ ]:
!mkdir images
!mkdir labels
!mkdir imagesReady
!mkdir annotations

In [42]:
from pathlib import Path

rey_folder = "/content/REY_roi_manualSelection1"
annotacions_folder = "/content/annotations"
imagesReady_folder = "/content/imagesReady"

def imageExits(imageName):
  dirImage = rey_folder + "/" + imageName
  my_file = Path(dirImage)  
  if my_file.is_file():
    return 1  
  return 0    

def annotationExits(imageName):
  imageNameForTxt = imageName.replace("png", "txt")
  imageNameForTxt = imageNameForTxt.replace("jpg", "txt")   
  dirImage = annotacions_folder + "/" + imageNameForTxt  
  my_file = Path(dirImage)  
  if my_file.is_file():
    return 1  
  return 0

In [28]:
def CheckBBValues(imageName, bb_xSuperiorIzq, bb_ySuperiorIzq, 
                  bb_xInferiorDerecho, bb_yInferiorDerecho, x_center, y_center, height, width):
  
  dirImage = rey_folder + "/" + imageName
  img1 = cv2.imread(dirImage)  
  
  cv2.circle(img1,(round(bb_xSuperiorIzq),round(bb_ySuperiorIzq)), 10, (0,0,255), -1)
  print("bb_xSuperiorIzq: {}, bb_ySuperiorIzq: {}".format(bb_xSuperiorIzq,bb_ySuperiorIzq))

  cv2.circle(img1,(round(bb_xInferiorDerecho),round(bb_yInferiorDerecho)), 10, (0,0,255), -1)
  plt.imshow(img1)
  plt.show()

  #print rectangles
  cv2.rectangle(img1,(round(bb_xSuperiorIzq),round(bb_ySuperiorIzq)),(round(bb_xInferiorDerecho),round(bb_yInferiorDerecho)),(0,255,0),3)
  plt.imshow(img1)
  plt.show()

  cv2.circle(img1,(x_center,y_center), 10, (0,0,255), -1)
  plt.imshow(img1)
  plt.show()

In [31]:
def GetBBDataForYOLO(imageName, bb_xSuperiorIzq, bb_ySuperiorIzq, bb_xInferiorDerecho, bb_yInferiorDerecho):
  dirImage = rey_folder + "/" + imageName  
  img1 = cv2.imread(dirImage)
  height, width = img1.shape[:2]

  #Get BB width and height
  b_width = (bb_xInferiorDerecho - bb_xSuperiorIzq)
  b_height = (bb_yInferiorDerecho - bb_ySuperiorIzq)
  #print("b_width: {}, b_height: {}".format(b_width,b_height))  

  #center point of bb
  x_center = round((bb_xInferiorDerecho + bb_xSuperiorIzq) / 2)
  y_center = round((bb_ySuperiorIzq + bb_yInferiorDerecho) / 2)

  #Check values --only for testing
  #CheckBBValues(imageName, bb_xSuperiorIzq, bb_ySuperiorIzq, bb_xInferiorDerecho, bb_yInferiorDerecho, x_center, y_center, height, width)
  
  #Normalizar valores para que estén entre 0 y 1
  xYolo = round(x_center / width, 6) 
  yYOLO = round(y_center / height, 6) 
  wYOLO = round(b_width / width, 6) 
  hYOLO = round(b_height / height, 6)

  return xYolo, yYOLO, wYOLO, hYOLO

In [46]:
#Utility function to move images 
from PIL import Image

def copyImage(file_name, destination_folder):   
  file_name_path = os.path.join(rey_folder, imageName) 
  img = Image.open(file_name_path)
  pathToSave = os.path.join(destination_folder, imageName)  
  img.save(pathToSave)

In [49]:
def CalculateAndSaveAnnotation(imageName, xYolo, yYOLO, wYOLO, hYOLO):  
  class_id = 0 
  #Save
  print_buffer = []
  # Name of the file which we have to save 
  imageNameForTxt = imageName.replace("png", "txt")
  imageNameForTxt = imageNameForTxt.replace("jpg", "txt") 

  save_file_name = os.path.join("/content/annotations", imageNameForTxt)

  #Write the bbox details to the file 
  print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, xYolo, yYOLO, wYOLO, hYOLO))  

  # Save the annotation to disk
  print("\n".join(print_buffer), file= open(save_file_name, "w"))
  

In [52]:
for index, row in dataframe.iterrows():
  name = row[0]
  xSuperiorLeft = row[1]
  ySuperiorLeft = row[2]
  xInferiorRight = row[3]
  yInferiorRight = row[4]
  
  imageName = name.strip()  

  #comprobar que existe la imagen en el folder  
  exists = imageExits(imageName)
  existsAnn = annotationExits(imageName)
  if(exists == 1 and existsAnn == 0):    
    xYolo, yYOLO, wYOLO, hYOLO = GetBBDataForYOLO(imageName, xSuperiorLeft, ySuperiorLeft, xInferiorRight, yInferiorRight)
    #print("xYolo: {}, yYOLO: {}, wYOLO: {}, hYOLO: {}".format(xYolo,yYOLO,wYOLO,hYOLO))
    CalculateAndSaveAnnotation(imageName, xYolo, yYOLO, wYOLO, hYOLO)
    copyImage(imageName, imagesReady_folder)
  else:
    print('error')  
  

In [53]:
# Read images and annotations
images = [os.path.join(imagesReady_folder, x) for x in os.listdir(imagesReady_folder)]
annotations = [os.path.join(annotacions_folder, x) for x in os.listdir(annotacions_folder)]
print(len(annotations))
print(len(images))

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

530
530


In [54]:
#Create the folders to keep the splits.
!mkdir images/train images/val images/test labels/train labels/val labels/test

In [55]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

In [56]:
# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')
move_files_to_folder(test_annotations, 'labels/test/')

In [ ]:
#Zip dataset
!zip -r '/content/images.zip' '/content/images/'
!zip -r '/content/labels.zip' '/content/labels/'

In [58]:
from google.colab import files
files.download('/content/images.zip')
files.download('/content/labels.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>